In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')
uniquePartsPath = os.path.join(homey, 'UniqueParts')

In [4]:
# Import possibly useful files
sys.path.insert(0, redouxPath)
import ForecastMain as fm
import ForecastTimelineBackend as ftlb
import ForecastSettings as fs
import xlsxwriter
import datetime as dt

In [7]:
# Get existing FB info
partsdf = fm.gather_parts()
invdf = fm.gather_inventory()
bomdf = fm.gather_boms()
descdf = pd.read_excel(os.path.join(rawDataPath, 'Descs.xlsx'), header=0)

In [50]:
# Placeholder for user input or iterative process
partRequest = '016-1077-10 r02'

In [12]:
fgLine = bomdf[(bomdf['PART'] == partRequest) & (bomdf['FG'] == 10)].copy()

In [37]:
bomNum = fgLine['BOM'].iloc[0]

In [38]:
bomdf[(bomdf['BOM'] == bomNum) & (bomdf['FG'] == 20)].copy()

,PART,BOM,QTY,FG,Make/Buy
6625,561-009-10,016-994-10 r01,16.0,20,Buy
6629,590-1312-10 r02,016-994-10 r01,2.0,20,Buy
6627,590-1313-10 r02,016-994-10 r01,1.0,20,Buy
6626,590-1314-10 r02,016-994-10 r01,1.0,20,Buy
6624,590-1315-10 r02,016-994-10 r01,1.0,20,Buy
6628,630-029-10,016-994-10 r01,16.0,20,Buy
9806,633-035-10,016-994-10 r01,16.0,20,Buy
6630,750-026-10 r01,016-994-10 r01,1.0,20,Make


In [39]:
bomNum

'016-994-10 r01'

In [40]:
### Returns a dataframe of the BOM that produces the FG of the input part
# bomdf: input bom exploder
# fgpart: input part number for FG of the BOM you want

def bom_return(bomdf, fgpart):
	fg = bomdf[bomdf['FG'] == 10].copy()
	# raw = bomdf[bomdf['FG'] == 20].copy()
	bomnum = fg['BOM'][fg['RAW'] == fgpart]
	if len(bomnum.index) > 0:
		bomnum = bomnum.iloc[0]
	else:
		bomnum = "nonononono"
	bomtemp = bomdf[bomdf['BOM'] == bomnum].copy()
	return bomtemp


### Returns multiplier necessary to run bom to match checkinv number
# singlebomdf: single FG bom exploder (comes from bom_return())
# checkinv: inventory number you want to product

def fg_to_multiplier(singlebomdf, checkinv):
	fgqty = singlebomdf['QUANTITY'][singlebomdf['FG'] == 10].copy()
	if len(fgqty.index) > 0:
		fgqty = fgqty.iloc[0]
	else:
		fgqty = 1
	multiplier = checkinv/fgqty
	return multiplier


### Returns BOM with quantities multiplied by an input number
# bomdf: input bom exploder format
# multi: number to multiply by - not sure if floats will work, we'll see soon enough

def bom_multiplier(bomdf, multi):
	bomdf['mult'] = bomdf['QUANTITY'].copy() * multi
	bomdf.drop('QUANTITY', axis=1, inplace=True)
	bomdf.rename(columns={'mult':'QUANTITY'}, inplace=True)
	return bomdf.copy()


### Returns a df with P/N, Qty, and BOM for FG:
# bomfind: input bomexploder
# fgsearch: input part number you want for the FG
# output eg:
# 		|RAW			|QUANTITY	|	BOM
# 2598	|016-450-10 r02	|1.0		|	016-450-10 r02


def find_fg(bomfind, fgsearch):
	findit = bomfind[bomfind['FG'] == 10].copy()
	singlefgdf = findit[['RAW','QUANTITY','BOM']][findit['RAW'] == fgsearch].copy()
	return singlefgdf



### Returns a list of parts on a BOM:
# allboms: input full bomexploder
# makebuy: input makebuy sheet with AVGCOST included
# exactpart: input the part number you wish to run

# output eg:
# 	|BOM				|QUANTITY	|PARTNUM			|Make/Buy	
# 0	|016-450-10 r02		|1.0		|745-025-10 r01		|Buy		
# 1	|016-450-10 r02		|1.0		|590-785-10			|Buy		
# 2	|016-450-10 r02		|1.0		|590-538-10 r01		|Buy		

# NOTE: Uses find_fg()!! 

def basic_bom_explode(allboms, makebuy, exactpart):
	# getting a df started with just the PN and qty of 1
	runlist = find_fg(allboms, exactpart)
	# declare some variables for the loop
	buylist = pd.DataFrame()
	this = 0
	limiter = len(runlist.index)
	while this < limiter:
	    partcheck = runlist['RAW'].iloc[this]
	    partqty = runlist['QUANTITY'].iloc[this]
	    tempbom = bom_return(allboms, partcheck)
	    tempmult = fg_to_multiplier(tempbom, partqty)
	    tempbom = bom_multiplier(tempbom.copy(), tempmult)
	    tempbom = pd.merge(tempbom.copy(), makebuy[['PARTNUM','Make/Buy']], left_on='RAW', right_on='PARTNUM')
	    tempbom = tempbom[tempbom['FG'] != 10].copy()
	    tempbuylist = tempbom[tempbom['Make/Buy'] == 'Buy'].copy()
	    temprunlist = tempbom[tempbom['Make/Buy'] == 'Make'].copy()
	    buylist = buylist.copy().append(tempbuylist.copy())
	    runlist = runlist.copy().append(temprunlist[['RAW','QUANTITY','BOM']])
	    # Added numbers to make increase the length of runlist
	    print('limiter is', str(limiter), 'and this is', str(this))
	    limiter = len(runlist.index)
	    this+=1
	    

	splaybom = buylist.copy().append(runlist.copy())

	### I'm adjusting this part to dodge "Make" parts that are missing BOMs ... The commented area is the original, in case this is a mistake
	# splaybom.drop(['FG','Make/Buy','PARTNUM'], axis=1, inplace=True)
	# splaybom.rename(columns={'RAW':'PARTNUM'}, inplace=True)
	# splaybom = pd.merge(splaybom.copy(), makebuy[['PARTNUM','Make/Buy']], on='PARTNUM', how='left')
	# return splaybom
	### End of the original set.  The following part will ignore dataframes with a length of 0.

	if len(splaybom.index) > 0:
		splaybom.drop(['FG','Make/Buy','PARTNUM'], axis=1, inplace=True)
		splaybom.rename(columns={'RAW':'PARTNUM'}, inplace=True)
		splaybom = pd.merge(splaybom.copy(), makebuy[['PARTNUM','Make/Buy']], on='PARTNUM', how='left')
		return splaybom
	else:
		return pd.DataFrame()



### drops BOM and Make/Buy off the output of basic_bom_explode and sums QTYs:

def sum_bom(thisbom):
	thisbom = thisbom.groupby('PARTNUM').sum().copy()
	thisbom = thisbom.reset_index().copy()
	return thisbom



### adds Make/Buy and AVGCOST to list of PN:

def add_mb_to_bom(thisbom, makebuy):
	thisbom = pd.merge(thisbom.copy(), makebuy, on='PARTNUM', how='left')
	return thisbom



### takes a "Make" part number and inventory level to unbuild it into raw goods and add it to an inventory list:


def get_raw_goods_out(invsheet, allboms, partnumhere, partinvhere):
	# the usual, find bom, multiply it by inv
	tempbom = bom_return(allboms, partnumhere)
	tempmult = fg_to_multiplier(tempbom, partinvhere)
	tempbom = bom_multiplier(tempbom, tempmult)
	# seperate the FG from the tempbom
	fgtemp = tempbom[tempbom['FG'] == 10].copy()
	tempbom = tempbom[tempbom['FG'] != 10].copy()
	# make the FG negative
	fgtemp['QUANTITY'] = fgtemp['QUANTITY'].copy() * (-1)
	# re attach FG to tempbom
	tempbom = tempbom.copy().append(fgtemp.copy())
	# taking the number and adding it to invsheet
	tempbom.rename(columns={'RAW':'PARTNUM','QUANTITY':'calc'}, inplace=True)
	invsheet = pd.merge(invsheet.copy(), tempbom[['PARTNUM','calc']], on='PARTNUM', how='left')
	invsheet['calc'] = invsheet['calc'].fillna(0).copy()
	invsheet['QTYONHAND'] = invsheet['QTYONHAND'].fillna(0).copy()
	invsheet['QTYONHAND'] = invsheet['QTYONHAND'].copy() + invsheet['calc']
	invsheet.drop('calc', axis=1, inplace=True)
	return invsheet.copy()


### runs a list to put through get_raw_goods_out()

def make_list_add_inv(listtomake, bominvsheet, bomex):
	bominvsheet = bominvsheet.copy()
	for this in listtomake.index:
		partcheck = listtomake['PARTNUM'].ix[this]
		partqty = listtomake['QTYONHAND'].ix[this]
		bominvsheet = get_raw_goods_out(bominvsheet, bomex, partcheck, partqty)
	return bominvsheet


### produces a list of make parts with current inv from a bom with inv included (insert part number of FG so it can be removed):

def make_list_from_bom(bomwithinv, fgproduct):
	makeonly = bomwithinv[bomwithinv['Make/Buy'] == 'Make'].copy()
	makeonly = makeonly[makeonly['PARTNUM'] != fgproduct].copy()
	makeonly.drop(['QUANTITY','Make/Buy','AVGCOST'], axis=1, inplace=True)
	makeonly.dropna(inplace=True)
	return makeonly


### experiment with only doing one bom level at a time:

### takes a "Make" part number and inventory level to unbuild it into raw goods and add it to an inventory list:
	# Also adds more to the runlist
def get_raw_goods_out_add_to_make_list(bominvsheet, allbplo, makebuydf, pnhere, pinvhere):
	# the usual, find bom, multiply it by inv
	temporarybom = bom_return(allbplo, pnhere)
	multiplybythis = fg_to_multiplier(temporarybom, pinvhere)
	temporarybom = bom_multiplier(temporarybom, multiplybythis)
	# seperate the FG from the temporarybom
	fingoodtemp = temporarybom[temporarybom['FG'] == 10].copy()
	temporarybom = temporarybom[temporarybom['FG'] != 10].copy()
	# save this for later to add make parts to run through list
	makecheck = temporarybom.copy()
	# make the FG negative
	fingoodtemp['QUANTITY'] = fingoodtemp['QUANTITY'].copy() * (-1)
	# re attach FG to temporarybom
	temporarybom = temporarybom.copy().append(fingoodtemp.copy())
	# taking the number and adding it to bominvsheet
	temporarybom.rename(columns={'RAW':'PARTNUM','QUANTITY':'calc'}, inplace=True)
	bominvsheet = pd.merge(bominvsheet.copy(), temporarybom[['PARTNUM','calc']], on='PARTNUM', how='left')
	bominvsheet['calc'] = bominvsheet['calc'].fillna(0).copy()
	bominvsheet['QTYONHAND'] = bominvsheet['QTYONHAND'].fillna(0).copy()
	bominvsheet['QTYONHAND'] = bominvsheet['QTYONHAND'].copy() + bominvsheet['calc']
	bominvsheet.drop('calc', axis=1, inplace=True)
	# want to do the make list now
	makecheck = pd.merge(makecheck.copy(), makebuydf[['PARTNUM','Make/Buy']].copy(), left_on='RAW', right_on='PARTNUM')
	makecheck = makecheck[makecheck['Make/Buy'] == 'Make'].copy()
	makecheck.drop(['RAW','BOM','FG','Make/Buy'], axis=1, inplace=True)
	return (bominvsheet.copy(), makecheck.copy())


###

# part_to_product_reference uses the "prepped" sheet and to find any part that is short and a "Make" part.
# Then it explodes the BOM for each of those parts and appends each of them to the same dataframe.

# Columns produced are:
# BOM : Each part that was used from the prep sheet
# Make/Buy : The Make/Buy state of the PARTNUM column
# PARTNUM : The part numbers for the raw goods on each of the BOMs
# QUANTITY : The amount of the PARTNUM used on said BOM


def part_to_product_reference(bomsheethere, makebuysheethere, prepsheethere, availcol):
	lowpostprep = prepsheethere[prepsheethere[availcol] < 0].copy()

	lowpostprep = lowpostprep[lowpostprep['Make/Buy'] == 'Make'].copy()

	reference = pd.DataFrame()
	for each in lowpostprep['PARTNUM']:
		bomex = basic_bom_explode(bomsheethere, makebuysheethere, each)
		bomex['BOM'] = each
		reference = reference.copy().append(bomex.copy())
	return reference.copy()

In [ ]:
def bom_by_multiplier(partRequest, multiplier, bomdf):
    

In [41]:
filepathbomex='Z:\planning systems\python versions\\bomex\\'
filepathcodes='Z:\planning systems\python versions\codes\\'
filepathreports='Z:\planning systems\Reports\BOM Exploder\\'




bplo = pd.read_excel(filepathcodes + 'bomexploder.xlsx', index=None, na_value=['NA'])
mb = pd.read_excel(filepathcodes + 'makebuy.xlsx', index=None, na_value=['NA'])
partid = pd.read_excel(filepathcodes + 'partid.xlsx', index=None, na_value=['NA'])
inv = pd.read_excel(filepathcodes + 'invoh.xlsx', index=None, na_value=['NA'])

In [51]:
bomexpl = basic_bom_explode(bplo, mb, partRequest)

bomexpl = sum_bom(bomexpl)
bomexpl = add_mb_to_bom(bomexpl, mb)

limiter is 1 and this is 0
limiter is 17 and this is 1
limiter is 21 and this is 2
limiter is 21 and this is 3
limiter is 21 and this is 4
limiter is 21 and this is 5
limiter is 21 and this is 6
limiter is 21 and this is 7
limiter is 21 and this is 8
limiter is 21 and this is 9
limiter is 21 and this is 10
limiter is 21 and this is 11
limiter is 21 and this is 12
limiter is 21 and this is 13
limiter is 21 and this is 14
limiter is 21 and this is 15
limiter is 21 and this is 16
limiter is 21 and this is 17
limiter is 23 and this is 18
limiter is 25 and this is 19
limiter is 25 and this is 20
limiter is 25 and this is 21
limiter is 25 and this is 22
limiter is 25 and this is 23
limiter is 25 and this is 24


In [62]:
bomdf.sort_values(by='PART', axis=0, inplace=True)

In [68]:
bomdf = bomdf[bomdf['FG'] == 20].copy()

In [71]:
bomdf.drop_duplicates(subset='PART', keep=False, inplace=True)

In [73]:
bomexpl = bomexpl[bomexpl['Make/Buy'] == 'Buy'].copy()

In [72]:
bomdf

,PART,BOM,QTY,FG,Make/Buy
3213,010-004-10,740-193-10,1.0,20,Buy
5908,010-005-10,016-905-10 r01,1.0,20,Buy
6937,010-542-10,010-596-10 r01,1.0,20,Buy
8749,010-543-10,022-211-10,1.0,20,Buy
22886,010-549-10,022-221-10 r01,2.0,20,Buy
8750,010-551-10,022-211-10,1.0,20,Buy
8759,010-560-10,022-211-10,1.0,20,Buy
8233,010-571-10,022-207-10 r01,3.0,20,Buy
2603,010-574-10,016-1071-10 r01,1.0,20,Buy
3390,010-575-10,016-1071-10 r01,1.0,20,Buy


In [75]:
for index, row in bomexpl.iterrows():
    if row['PARTNUM'] in bomdf['PART'].unique():
        print(row['PARTNUM'])

130-026-10
151-035-10
151-037-10
151-094-10
151-112-10
151-124-10
194-002-10
330-013-10
360-205-10
360-206-10
361-002-10
367-019-10
410-002-10
421-011-10
521-000-10
700-096-10 r07
700-114-10 r07
700-389-10 r02


In [74]:
bomexpl

,PARTNUM,QUANTITY,Make/Buy,AVGCOST
0,012-002-10,1.00000,Buy,10.498115
5,030-996-10 r01,1.00000,Buy,3.492500
7,110-132-10,1.00000,Buy,0.012739
8,110-133-10,2.00000,Buy,0.010504
9,115-007-10,1.00000,Buy,0.046272
10,130-026-10,2.00000,Buy,0.684682
11,148-053-10,1.00000,Buy,51.816654
12,151-027-10,29.00000,Buy,0.003978
13,151-035-10,4.00000,Buy,0.006290
14,151-037-10,2.00000,Buy,0.004200
